In [3]:
# def day18_parser(line: str) -> tuple:
#     pass
    

# def day18_part1(policies: list[tuple]) -> int:
#     pass


# def day18_part2(policies: list[tuple]) -> int:
#     pass

# run(day=18, parser=day18_parser, sep='\n', funcs=[day18_part1, day17_part2], example=True, actual=False)

# for i in range(1, 26):
#     with open(f'data/day{i:02}.txt', 'a') as f:
#         f.write('')

In [4]:
import timeit
from itertools import permutations, combinations, product, chain, islice
from math import ceil, floor, gcd

#Dict, Tuple, Set, List, Iterator, Optional, Union, Sequence

# from __future__  import annotations
from collections import defaultdict, namedtuple, deque, Counter
# from itertools   import permutations, combinations, product, chain
from functools   import partial, lru_cache, reduce 
from typing import Any, Callable, Optional, NamedTuple, Generator # Iterator, Optional, Union, Sequence, DefaultDict?
# from contextlib  import contextmanager

# import operator
# import math
# import ast
# import sys
import re


from more_itertools import minmax



cat = ''.join


def first(iterable, default=None) -> object:
    "Return first item in iterable, or default."
    return next(iter(iterable), default)


def quantify(iterable, pred=bool) -> int:
    "Count the number of items in iterable for which pred is true."
    return sum(1 for item in iterable if pred(item))


def lines(text: str) -> list[str]:
    "Split the text into a list of lines."
    return text.strip().splitlines()

In [5]:
def data(path: str, parser: Callable[[str], Any]=str, sep: str='\n') -> list:
    "Load input textfile into sections separated by `sep`, and apply `parser` to section." 
    assert isinstance(path, str), path
    with open(path, 'r') as f:
        if sep is None and parser is None:  # raw text as string
            return f.read() 
        sections = f.read().rstrip().split(sep)
        return list(map(parser, sections))


def run(day: int, funcs: list=[], example: bool=True, actual: bool=False, parser=str, sep: str='\n') -> None:
    """Run a day's example (and/or actual) input on a list of callables for each part i.e. [part1, part2]. 
    While working can use [part1, lambda _: None]."""
    if example:
        puzzle_input = data(path=f'data/example/day{day:02}.txt', parser=parser, sep=sep)
        print(f'{"Part 1 (example): " + str(funcs[0](puzzle_input)):<25}', end='\t')
        print(f'{"Part 2 (example): " + str(funcs[1](puzzle_input)):<25}', end='\n')
    
    if actual:
        puzzle_input = data(path=f'data/day{day:02}.txt', parser=parser, sep=sep)
        print(f'{"Part 1 (actual): " + str(funcs[0](puzzle_input)):<25}', end='\t')
        print(f'{"Part 2 (actual): " + str(funcs[1](puzzle_input)):<25}', end='\n')


def test_alternatives(day: int, funcs: list=[], runs: int=10, parser=str, sep: str='\n') -> None:
    "Test alternative approaches on a specific part of a day's actual input."
    puzzle_input = data(path=f'data/day{day:02}.txt', parser=parser, sep=sep)
    
    print('Func Result:', end='\t')
    for func in funcs: 
        print(func(puzzle_input), end='  ')

    print('\nFunc Runtime:', end='\t')
    for func in funcs:  # Compare each functions runtime
        print(f'{timeit.timeit("func(puzzle_input)", number=runs, globals=locals()):.3f}', end='  ')

# Done

## Day 01: Report Repair
- Part 1: find two numbers that sum to 2020, return their product
- Part 2: find three numbers that sum to 2020, return their product

In [6]:
def day1_part1(nums: list[int]) -> int:
    for x in nums:
        for y in nums:
            if x + y == 2020:
                return x * y


def day1_part2(nums: list[int]) -> int:
    for x in nums:
        for y in nums:
            for z in nums:
                if x + y + z == 2020:
                    return x * y * z


run(day=1, parser=int, sep='\n', funcs=[day1_part1, day1_part2], example=False, actual=True)

Part 1 (actual): 802011  	Part 2 (actual): 248607374


In [7]:
def day1_part1b(nums: list[int]) -> int: # still O(n^2) as above, but slighty faster
    for i, x in enumerate(nums):
        for y in islice(nums, i): # not using nums[i:] as that creates a copy each time (would make this O(n^3))
            if x + y == 2020:
                return x * y


def day1_part1c(nums: list[int]) -> int: # O(n^2)
    for x, y in combinations(nums, 2): 
        if x + y == 2020:
            return x * y


def day1_part1d(nums: list[int]) -> int: # O(n)
    seen = set()
    for x in nums:
        if (2020 - x) in seen:
            return (2020 - x) * x
        seen.add(x)


def day1_part1e(nums: list[int]) -> int: # O(n) the second loop is only run max twice (i.e. 2020 + 20 and 20 + 2020)
    nums = set(nums) # ideally input would already be a set
    return first(
        x * y 
        for x in nums 
        for y in nums.intersection({2020 - x})
        if x != y
        )


test_alternatives(day=1, funcs=[day1_part1, day1_part1b, day1_part1c, day1_part1d, day1_part1e], runs=1000, parser=int)

Func Result:	802011  802011  802011  802011  802011  
Func Runtime:	0.304  0.143  0.325  0.006  0.003  

In [8]:
def day1_part2b(nums: list[int]) -> int: # still O(n^3) as above, has faster wall time though
    for x, y, z in combinations(nums, 3): 
        if x + y + z == 2020:
            return x * y * z


def day1_part2c(nums: list[int]) -> int: # O(n^2), but slower wall time (on the test input/size)
    set_nums = set(nums)
    for x, y in combinations(nums, 2):
        if (2020 - x - y) in set_nums:
            return x * y * (2020 - x - y)


def day1_part2d(nums: list[int]) -> int: # O(n^2)
    nums = set(nums)
    return first(
        x * y * z 
        for x, y in combinations(nums, 2) 
        for z in nums.intersection({2020 - x - y})
        if x != y != z
        )


def day1_part2e(nums: list[int]) -> int: # O(N^2) time, O(1) space
    nums = sorted(nums)  # sort then use two pointers
    for i in range(len(nums)):
        l, r = i+1, len(nums)-1 # left starts at next index, right starts at last index
        while l < r:
            if (nums[l] + nums[r] + nums[i]) > 2020:
                r -= 1
            elif (nums[l] + nums[r] + nums[i]) < 2020:
                l += 1
            elif (nums[l] + nums[r] + nums[i]) == 2020:
                return nums[l] * nums[r] * nums[i]


test_alternatives(day=1, funcs=[day1_part2, day1_part2b, day1_part2c, day1_part2d, day1_part2e], runs=10, parser=int)

Func Result:	248607374  248607374  248607374  248607374  248607374  
Func Runtime:	1.331  0.483  0.007  0.005  0.001  

## Day 02: Password Philosophy
- Input Line: #-# character: password (i.e. 1-3 e: abcde)
- Part 1: find number of passwords with character count between low and high # 
- Part 2: find number of passwords with character at idx #(1) or idx #(2) (1-based indexes and XOR)

In [9]:
def day2_parser(line: str) -> tuple:
    "Given line '1-3 e: mdie', return (1, 3, 'e', 'mdie')"
    nums, char, pw = line.split()
    a, b = map(int, nums.split('-'))
    char = char.rstrip(':')
    return (a, b, char, pw)
    

def day2_part1(policies: list[tuple]) -> int:
    cnt = 0
    for policy in policies:
        lo, hi, char, pw = policy
        if lo <= pw.count(char) <= hi:
            cnt += 1
    return cnt


def day2_part2(policies: list[tuple]) -> int:
    cnt = 0
    for policy in policies:
        pos1, pos2, char, pw = policy
        one, two = pw[pos1-1], pw[pos2-1] # minus 1 as the positions are 1-based indexed
        if (one==char) != (two==char): # could use xor (^) instead
            cnt += 1
    return cnt


run(day=2, parser=day2_parser, sep='\n', funcs=[day2_part1, day2_part2], example=True, actual=True)

Part 1 (example): 2      	Part 2 (example): 1      
Part 1 (actual): 477     	Part 2 (actual): 686     


In [10]:
Policy = tuple[int, int, str, str] 
Policies = list[Policy]


def day2_parserb(line: str) -> Policy:
    "Given line '1-3 e: mdie', return (1, 3, 'e', 'mdie')"
    a, b, char, pw = re.findall(r'[^-:\s]+', line) # or r'(\d+)-(\d)+ (.): (.+)$'
    return (int(a), int(b), char, pw)


def day2_part1b(policies: Policies) -> int:
    def valid_password(policy: Policy) -> bool:
        lo, hi, char, pw = policy
        return lo <= pw.count(char) <= hi
    return quantify(policies, valid_password)


def day2_part2b(policies: Policies) -> int:
    def valid_password(policy: Policy) -> bool:
        pos1, pos2, char, pw = policy
        one, two = pw[pos1-1], pw[pos2-1]  # minus 1 as these positions are 1-based indexed
        return (one==char) ^ (two==char)  # could use != instead of XOR
    return quantify(policies, valid_password)


run(day=2, parser=day2_parserb, sep='\n', funcs=[day2_part1b, day2_part2b], example=True, actual=True)

Part 1 (example): 2      	Part 2 (example): 1      
Part 1 (actual): 477     	Part 2 (actual): 686     


## Day 3: Toboggan Trajectory
- Input is a map of freespaces (.) & trees (#) copied infinitely to the right
- Part 1: count trees landed on moving right then down (3,1 places starting from top left)
- Part 2: return product of tree counts on the following paths: (1,1), (3,1), (5,1), (7,1), (2,1)

In [11]:
def day3_helper(treemap: list[str], x_step: int, y_step: int):
    x = 0  # starting horizontal position
    y = 0  # starting vertical position
    tree_count = 0
    
    while y < len(treemap):
        row = treemap[y]
        if row[x] == '#':  # landed on a tree
            tree_count += 1
        x = (x + x_step) % len(row)  # wrap around the x-axis
        y += y_step  # move down the y-axis
    return tree_count


def day3_part1(treemap: list[str]) -> int:
    return day3_helper(treemap, 3, 1)


def day3_part2(treemap: list[str]) -> int:
    f = lambda x, y: day3_helper(treemap, x, y)
    return f(1, 1) * f(3, 1) * f(5, 1) * f(7, 1) * f(1, 2)


run(day=3, parser=str, sep='\n', funcs=[day3_part1, day3_part2], example=True, actual=True)

Part 1 (example): 7      	Part 2 (example): 336    
Part 1 (actual): 191     	Part 2 (actual): 1478615040


In [12]:
def day3_helperb(treemap: list[str], x_step=3, y_step=1) -> int:
    return quantify(row[(x_step * y) % len(row)] == '#'
                    for y, row in enumerate(treemap[::y_step]))
    # aka:
    # cnt = 0
    # for y, row in enumerate(treemap[::y_step]):
    #     # x = (x_step * y) 
    #     # pos = (x_step * y) % len(row)
    #     # print(x, y, pos)
    #     cnt += 1 if row[(x_step * y) % len(row)] == '#' else 0
    # return cnt 

def day3_part1b(treemap: list[str]) -> int:
    return day3_helperb(treemap, 3, 1)


def day3_part2b(treemap: list[str]) -> int:
    f = lambda x, y: day3_helperb(treemap, x, y)
    return f(1, 1) * f(3, 1) * f(5, 1) * f(7, 1) * f(1, 2)


run(day=3, parser=str, sep='\n', funcs=[day3_part1b, day3_part2b], example=True, actual=True)

Part 1 (example): 7      	Part 2 (example): 336    
Part 1 (actual): 191     	Part 2 (actual): 1478615040


## Day 4: Passport Processing
- Input is a series of passports seperated by double newlines, each passort has # fields (key:value pairs).
- Part 1: count how many passports have required fields.
- Part 2: count how many passports have required fields and ensure each field meets set criteria.

In [13]:
REQUIRED_FIELDS = {"byr", "ecl", "eyr", "hcl", "hgt", "iyr", "pid"}


def day4_parser(line: str) -> dict:
    """Create a dict from text ala 'ecl:gry eyr:2020' -> {'ecl':'gry', 'eyr':'2020'}"""
    return dict(kv.split(":") for kv in line.split())


def day4_part1(passports: list[dict]) -> int:
    # alt.: set(p).issuperset(REQUIRED_FIELDS)
    return quantify(passports, lambda p: p.keys() >= REQUIRED_FIELDS)


def day4_part2(passports: list[dict]) -> int: 
    between = lambda v, start, end: start <= int(v) <= end
    height = lambda v: (v.endswith("cm") and between(v[:-2], 150, 193)) or (
        v.endswith("in") and between(v[:-2], 59, 76)
    )
    
    eye_colour = lambda v: v in {"amb", "blu", "brn", "gry", "grn", "hzl", "oth"}
    hair_colour = lambda v: re.match("#[0-9a-f]{6}$", v)
    passport_id = lambda v: re.match("[0-9]{9}$", v)

    def valid_passport(p) -> bool:
        return p.keys() >= REQUIRED_FIELDS and all(
            (
                between(p["byr"], 1920, 2002),
                between(p["iyr"], 2010, 2020),
                between(p["eyr"], 2020, 2030),
                height(p["hgt"]),
                eye_colour(p["ecl"]),
                hair_colour(p["hcl"]),
                passport_id(p["pid"]),
            )
        )

    return quantify(passports, valid_passport)



run(day=4, parser=day4_parser, sep="\n\n", funcs=[day4_part1, day4_part2], example=True, actual=True)

Part 1 (example): 2      	Part 2 (example): 2      
Part 1 (actual): 170     	Part 2 (actual): 103     


## Day 5: Binary Boarding
- Input is a list of seat ids of the format 'FFFBFFF RRL' (front/back & left/right) -> 357
- Part 1: find the maximum seat id
- Part 2: find the missing seat id (in the list of contiguous seat ids)

![""](img/day05.png "day5 example")

In [14]:
def binary_partitioning(search_space: str, binary_choice):
    lb, ub = 0, (2**len(search_space))-1  # lower and upper bounds
    for item in search_space:
        if item != binary_choice:
            lb = ceil(lb/2) + ceil(ub/2)
        else:
            ub = floor(lb/2) + floor(ub/2)
    return min(lb, ub)


def day5_helper(seats: list[str]) -> list[int]:
    "Convert a list of seat strings to seat ids ala ['FBFBBFFRLR'] -> [357]."
    row = partial(binary_partitioning, binary_choice='F')
    col = partial(binary_partitioning, binary_choice='L')
    return [row(seat[:7]) * 8 + col(seat[7:]) for seat in seats]


def day5_part1(seats: list[str]) -> int:
    return max(day5_helper(seats))


def day5_part2(seats: list[int]) -> set:
    seat_ids = day5_helper(seats)
    all_seat_ids = set(i for i in range(*minmax(seat_ids)))
    return (all_seat_ids - set(seat_ids))


run(day=5, parser=str, sep='\n', funcs=[day5_part1, day5_part2], example=True, actual=True)

Part 1 (example): 357    	Part 2 (example): set()  
Part 1 (actual): 888     	Part 2 (actual): {522}   


In [15]:
def day5_helperb(seats: list[str], table=str.maketrans('FLBR', '0011')) -> list[int]:
    "Treat a seat description as a binary number; convert to int."
    return [int(s.translate(table), base=2) for s in seats]

def day5_part1(seats: list[str]) -> int:
    return max(day5_helperb(seats))


def day5_part2(seats: list[int]) -> set:
    seat_ids = day5_helperb(seats)
    all_seat_ids = set(i for i in range(*minmax(seat_ids)))
    return (all_seat_ids - set(seat_ids))


run(day=5, parser=str, sep='\n', funcs=[day5_part1, day5_part2], example=True, actual=True)

Part 1 (example): 357    	Part 2 (example): set()  
Part 1 (actual): 888     	Part 2 (actual): {522}   


## Day 6: Custom Customs
- Input is groups (seperated by \n\n) of answers (one line per person in the group).
- Part 1: Sum the count of unique answers per group.
- Part 2: Sum the count of duplicate answers per group.

In [16]:
def day6_part1(groups: list[list[str]]) -> int:
    return sum(len(set(cat(g))) for g in groups)


def day6_part2(groups: list[list[str]]) -> int:
    return sum(len(set.intersection(*map(set, g))) for g in groups)


run(day=6, parser=lines, sep='\n\n', funcs=[day6_part1, day6_part2], example=True, actual=True)

Part 1 (example): 11     	Part 2 (example): 6      
Part 1 (actual): 6291    	Part 2 (actual): 3052    


## Day 7: Handy Haversacks
- Input is a mapping of bag -> bags it may contain (i.e. silver -> 2 bronze bags).
- Part 1: Find the count of bags that can (eventually) contain a 'shiny gold' bag.
- Part 2: Find how many bags are contained by a 'shiny gold' bag.

In [17]:
RULE_RE = re.compile(r'^(\w+ \w+) bags contain (.+)$')
CHILD_RE = re.compile(r'(\d+) (\w+ \w+)')


def day7_part1(rules: list[str]) -> int:
    def parse(rules: list[str]) -> defaultdict[str, list[str]]:
        """Returns a {child: [parent, ...]} mapping i.e. {'shiny gold: ['bright white', ...]}"""
        child_parents = defaultdict(list)
        
        for rule in rules:
            rule_match = RULE_RE.match(rule)
            parent, rest_of_line = rule_match.groups()
            
            for num, child in CHILD_RE.findall(rest_of_line):
                child_parents[child].append(parent)
        return child_parents
    
    def find_parents(child: str, parents: defaultdict[str, list[str]]) -> set[str]:
        """Returns a set of all parents for a given child (colour), inc. itself."""
        ret = {child}
        for parent in parents[child]:
            ret |= find_parents(parent, parents)
        return ret
    
    # Recursive depth-first search 
    # i.e. using a stack (here the function call stack) to visit each leaf node
    return len(find_parents('shiny gold', parse(rules)) - {'shiny gold'})

    ## Iterative
    # child_parents = parse(rules)
    # seen = set()
    # todo = ['shiny gold']  # using list as a stack
    
    # while todo:
    #     colour = todo.pop()
    #     seen.add(colour)
    #     todo.extend(child_parents[colour])
    # return len(seen - {'shiny gold'})


def day7_part2(rules: list[str]) -> int:
    def parse(rules: list[str]) -> dict[str, tuple[int, str]]:
        "Returns a {parent: [(num, child), ...]} mapping i.e. {'shiny gold: [(1, 'Bright White'), ...]}"
        parents_can_contain = defaultdict(list)  
        
        for rule in rules:
            rule_match = RULE_RE.match(rule)
            parent, rest_of_line = rule_match.groups()
            
            children = [(int(num), child) for num, child in CHILD_RE.findall(rest_of_line)]
            parents_can_contain[parent] = children
        return parents_can_contain

    def find_bag_count(n: int, child: str, parents: dict[str, tuple[int, str]]) -> int:
        """Returns a set of all parents for a given child (colour), inc. itself."""
        ret = n
        for child_n, c in parents[child]:
            ret += find_bag_count(n * child_n, c, parents)
        return ret
    
    # Recursive depth-first search 
    return find_bag_count(1, 'shiny gold', parse(rules)) - 1


run(day=7, parser=str, sep='\n', funcs=[day7_part1, day7_part2], example=True, actual=True)

Part 1 (example): 4      	Part 2 (example): 32     
Part 1 (actual): 332     	Part 2 (actual): 10875   


## Day 8: Handheld Halting
- Input is a list of assembly code instructions - no operation/accumulate(to a single global var)/jump(like goto) with a int value i.e. nop +0 / jmp +4 / acc -1.
- Part 1: find the first repeated instruction (indicates an endless loop) and return the accumulated value.
- Part 2: find the buggy instruction, by replacing 1 nop with a jmp OR vice versa, the program should end at -> num-lines + 1. Return the accumulated value.


In [18]:
def day8_parser(line: str) -> tuple:
    operation, argument = line.split()
    return (operation, int(argument)) 


def day8_helper(instructions: list[tuple]) -> tuple[int, int, set[int]]:
    accumulator = current_idx = 0
    seen = set()
    
    while current_idx not in seen and current_idx != len(instructions):
        seen.add(current_idx)
        op, arg = instructions[current_idx]
        
        if op == 'acc':
            accumulator += arg
            current_idx += 1
        elif op == 'nop':
            current_idx += 1
        elif op == 'jmp':
            current_idx += arg
        else:
            AssertionError(f"Unknown instruction: {op}")
    return accumulator, current_idx, seen


def day8_part1(instructions: list[tuple]) -> int:
    return day8_helper(instructions)[0]


def day8_part2(instructions: list[tuple]) -> int:
    # skipping over instructions not used: (4.69 / 1.34) = 3.5 times faster
    possible_bad_instructions = day8_helper(instructions)[2]

    flip = {'nop': 'jmp', 'jmp': 'nop'}
    for idx, (op, arg) in enumerate(instructions):
        if idx in possible_bad_instructions:
            if op in flip.keys():
                flipped_instructions = instructions[:]
                flipped_instructions[idx] = (flip[op], arg)
                accumulator, current_idx, seen = day8_helper(flipped_instructions)
                if current_idx == len(instructions):
                    return accumulator



run(day=8, parser=day8_parser, sep='\n', funcs=[day8_part1, day8_part2], example=True, actual=True)

Part 1 (example): 5      	Part 2 (example): 8      
Part 1 (actual): 1709    	Part 2 (actual): 1976    


## Day 9: Encoding Error
- Input is a series of numbers. Each number x is proceeded by 25 (5 in example) nums, two of which sum to match x.
- Part 1: find the weird number (lacks any preceding combination of numbers that sum to match).
- Part 2: find the

In [19]:
def day9_part1(nums: list[int]) -> int: # O(n)
    def summed_combinations(nums: list[int], r: int=2) -> int:
        for pair in combinations(nums, r=2):
            yield sum(pair)
            
    skip_n = 25 if len(nums) > 20 else 5  # handle the smaller example sliding window

    for idx, num in enumerate(nums):
        if idx >= skip_n:
            # summed_pairs = {sum(pair) for pair in combinations(nums[idx:idx+25], 2)}
            summed_pairs = summed_combinations(nums[idx-skip_n:idx])
            if num not in summed_pairs:
                return num


def day9_part2(nums: list[int]) -> int:  # O(n^3)
    target = day9_part1(nums)
    for i in range(len(nums)):
        for j in range(i+1, len(nums)):
            contiguous_set = nums[i:j+1]
            if sum(contiguous_set) == target:
                return sum(minmax(contiguous_set))


run(day=9, parser=int, sep='\n', funcs=[day9_part1, day9_part2], example=True, actual=True)

Part 1 (example): 127    	Part 2 (example): 62     
Part 1 (actual): 26134589	Part 2 (actual): 3535124 


In [20]:
# 22.3 ms ± 129 µs per loop (mean ± std. dev. of 7 runs, 10 loops each) # set comp
# 18.4 ms ± 36.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each) # list comp
# 4.18 ms ± 21.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each) # generator function

In [21]:
def day9_part1b(nums: list[int]) -> int:
    for i in range(25, len(nums)):
        prev = nums[i-25:i]
        for x, y in combinations(prev, 2):
            if x + y == nums[i]:
                break
        else:
            return nums[i]


def day9_part1c(nums, p=25):
    """Find the first number in the list of numbers (after a preamble of p=25 numbers) 
    which is not the sum of two of the p numbers before it."""
    twosums = lambda nums: map(sum, combinations(nums, 2))
    return first(x for i, x in enumerate(nums) 
                # if i > p skips first 25, avoiding (by boolean short-circuiting) an index error on the sliced list
                if i > p and x not in twosums(nums[i-p:i]))  




test_alternatives(day=9, funcs=[day9_part1, day9_part1b, day9_part1c], runs=100, parser=int)

Func Result:	26134589  26134589  26134589  
Func Runtime:	0.193  0.108  0.115  

In [22]:
def day9_part2b(nums: list[int]) -> int:  # O(n)
    """Sliding window approach."""
    start = end = 0
    total = nums[0]
    size = len(nums)
    target = day9_part1(nums)
    
    while start < size:
        if total < target:
            end = end + 1 if end < size else size
            total += nums[end]
        elif total > target:
            total -= nums[start]
            start += 1
        else:  # total == target
            window = nums[start:end+1]
            return sum(minmax(window))


def day9_part2c(nums):
    "Find a contiguous subsequence of nums that sums to target; add their max and min."
    target = day9_part1(nums)
    subseq = find_subseq(nums, target)
    return max(subseq) + min(subseq)


def find_subseq(nums, target) -> Optional[deque]:
    "Find a contiguous subsequence of nums that sums to target."
    subseq = deque()
    total = 0
    for num in nums:
        if total < target:
            subseq.append(num)
            total += num
        if total == target and len(subseq) >= 2:
            return subseq
        while total > target:
            total -= subseq.popleft()
    return None


test_alternatives(day=9, funcs=[day9_part2, day9_part2b, day9_part2c], runs=10, parser=int)

Func Result:	3535124  3535124  3535124  
Func Runtime:	6.025  0.019  0.019  

## Day 10: Adapter Array
- Input is a list of integers representing "joltages" of adapters. Starting from 0 connect these (can combine those -1, -2, -3 joltage apart) to power your device (with a joltage +3 max(joltages) input).

- Part 1: Using all adapters, find the number of 3-gap and 1-gap devices and return the product of these counts.
- Part 2: Count the number of possible arrangements of the adapters.

![""](img/day10.png "day10 example")

In [23]:
def day10_part1(joltages: list[int]) -> int:
    joltages = sorted(joltages)  # O(N*log(N))
    joltages.insert(0, 0)
    diff1 = 0 # 1 jolt differences cnt
    diff3 = 1 # 3 jolt differences cnt (inc. your device as a +3 gap)

    for idx in range(len(joltages) - 1):
        val, next_val = joltages[idx], joltages[idx + 1]
        if next_val - val == 1:
            diff1 += 1
        elif next_val - val == 3:
            diff3 += 1
    return diff1 * diff3


def day10_part2(joltages: list[int]) -> int:
    @lru_cache  # due to caching this ends up O(1)
    def streak_combs(n: int) -> int:
        if n in {0, 1}:
            return 1
        elif n == 2:
            return 2
        return streak_combs(n - 1) + streak_combs(n - 2) + streak_combs(n - 3) 
    
    joltages = sorted(joltages)
    total = 1
    streak = prev = 0
    
    for joltage in joltages:
        if joltage == prev + 1:
            streak += 1
        else:
            #print(f'{streak=}')
            total *= streak_combs(streak)
            streak = 0
        prev = joltage
    total *= streak_combs(streak)
    return total

run(day=10, parser=int, sep='\n', funcs=[day10_part1, day10_part2], example=True, actual=True)

Part 1 (example): 220    	Part 2 (example): 19208  
Part 1 (actual): 2176    	Part 2 (actual): 18512297918464


In [24]:
def day10_part1b(joltages: list[int]) -> int:
    joltages = sorted(joltages)
    counts = Counter({3: 1})
    current = 0
    for joltage in joltages:
        counts[joltage - current] += 1
        current = joltage
    return counts[3] * counts[1]


test_alternatives(day=10, funcs=[day10_part1, day10_part1b], runs=1, parser=int)

Func Result:	2176  2176  
Func Runtime:	0.000  0.000  

In [25]:
def day10_part2b(joltages: list[int]) -> int:
    combs = defaultdict(int, {0: 1})  # use callable int() as == 0
    joltages = sorted(joltages)
    
    for j in joltages:
        combs[j] = combs[j - 1] + combs[j - 2] + combs[j - 3]
    return combs[joltages[-1]]


test_alternatives(day=10, funcs=[day10_part2, day10_part2b], runs=10, parser=int)

Func Result:	18512297918464  18512297918464  
Func Runtime:	0.000  0.000  

## Day 11
Input is a map (matrix) of strings representing seat occupancy (L Open Seat, # Occuupied Seat, . aisle).

- Part 1: return the sum of occupied seats, after applying the following rules for each seat simultaneously (batch) until the map settles / stops changing:
  - count the number of occupied seats (for each seat's surrounding 8 seats)
  - for L: if there are 0 occupied seats change to #
  - for #: if there are >= 4 occupied seats change to L
  - else no change
- Part 2: return the sum of occupied seats, after applying the following rules for each seat simultaneously (batch) until the map settles / stops changing:
  - count the number of occupied seats (for each seat's surrounding seats outwards infinitely / bounded only by the map size)
  - for L: if there are 0 occupied seats change to #
  - for #: if there are >= 5 occupied seats change to L
  - else no change

In [26]:
def parse_seats(lines: list[str]) -> dict[tuple[int, int], set[tuple[int, int]]]:
    def generate_paths(w: int, h: int):
        for y in range(0, h):  # to the right
            yield ((x, y) for x in range(w))
        for x in range(0, w):  # down
            yield ((x, y) for y in range(h))
        for x0 in range(-h + 1, w):  # down and to the right
            yield ((x0 + y, y) for y in range(max(0, -x0), min(h, w - x0)))
        for x0 in range(0, w + h):  # down and to the left
            yield ((x0 - y, y) for y in range(max(0, x0 - w + 1), min(h, x0 + 1)))

    # find all seats in the input, and their connections.
    w, h = len(lines[0]), len(lines)
    seats = {(x, y): {(x, y)} for x in range(w) for y in range(h) if lines[y][x] != '.'}
    for path in generate_paths(w, h):
        previous = None
        for current in path:
            if current in seats:
                if previous:
                    seats[previous].add(current)
                    seats[current].add(previous)
                previous = current
    return seats


def solve(seats: dict[tuple[int, int], set[tuple[int, int]]], n: int):
    # assume that all seats are free at the beginning.
    free = {seat for seat, visible in seats.items() if len(visible) > n} # after 2 iterations
    while True:
        changed = set(seat for seat, visible in seats.items() if visible <= free or seat not in free and len(visible - free) > n)
        if not changed:
            break
        free ^= changed
    return len(seats) - len(free)


def day11_part1(seats: list[str]) -> int:
    seats = {(i, j): {(p, q) for (p, q) in visible if max(abs(i-p), abs(j-q)) < 2} for (i, j), visible in parse_seats(seats).items()}
    return solve(seats, 4)


def day11_part2(seats: list[str]) -> int:
    seats = parse_seats(seats)
    return solve(seats, 5)


run(day=11, parser=str, sep='\n', funcs=[day11_part1, day11_part2], example=True, actual=True)

Part 1 (example): 37     	Part 2 (example): 26     
Part 1 (actual): 2481    	Part 2 (actual): 2227    


## Day 12
- Input consists of one line (arrival timestamp) and one line of bus int ids ('x' means out of service). Each bus id also details the departure times (i.e. bus id 5 departs at time 0, 5, 10, ...)

- Part 1: find the first bus id that departs after our arrival time. Return the product of the bus id and time after arrival.
- Part 2: return the first timestamp of the first sequence of timestamps such that:
  - bus 0 appears at T
  - bus 1 appears at T + 1 (if not out of service)
  - bus n apeears at T + n (if not out of service)

In [27]:
# !grep -E '[LR]' data/day12.txt | sort | uniq -c

def day12_parser(line: str) -> tuple:
    action, value = re.match(r"([NESWLRF])(\d+)", line).groups()
    return (action, int(value))


def day12_part1(actions: tuple[str, int]) -> int:
    directions = ('N', 'E', 'S', 'W')
    course = {d: 0 for d in directions}
    current_direction = 'E'
    
    for act, val in actions:
        if act in directions:
            course[act] += val
        elif act in {'F'}:
            course[current_direction] += val
        else: # act in {'L', 'R'}
            offset = 1 if act == 'L' else -1
            magnitude = val // 90  # convert val (90, 180, 280) to (1, 2, 3)
            idx = directions.index(current_direction) - (offset * magnitude) % len(directions)
            current_direction = directions[idx]
            
    east_west = course['E'] - course['W']
    north_south = course['N'] - course['S']
    return abs(east_west) + abs(north_south)

def day12_part2(actions: tuple[str, int]) -> int:
    shp_y, shp_x = 0, 0
    wpt_y, wpt_x = 1, 10
    
    for act, val in actions:
        if   act == 'N': wpt_y += val
        elif act == 'S': wpt_y -= val
        elif act == 'E': wpt_x += val
        elif act == 'W': wpt_x -= val
        elif act == 'L':
            for _ in range(val // 90):
                wpt_y, wpt_x = wpt_x, wpt_y
                wpt_x *= -1
        elif act == 'R':
            for _ in range(val // 90):
                wpt_y, wpt_x = wpt_x, wpt_y
                wpt_y *= -1
        else:  # act == 'F'
            shp_y += val * wpt_y
            shp_x += val * wpt_x    

    return abs(shp_y) + abs(shp_x)


run(day=12, parser=day12_parser, sep='\n', funcs=[day12_part1, day12_part2], example=True, actual=True)

Part 1 (example): 25     	Part 2 (example): 286    
Part 1 (actual): 2270    	Part 2 (actual): 138669  


## Day 13
- Input is set of actions (NESW / LR / F) and magnitudes (or degrees).

- Part 1: follow rules to move the ship X units either NESW (or forward in the direction faced (starts facing east)) or rotate the ships caridnal direction (RL 90/180/270 degrees), return the manhattan distance from origin.
- Part 2: NESW and RL now applies to a waypoint beacon, F moves the ship in the direction of that becon X * the waypoints relative direction to the ship, return the manhattan distance from origin.

In [28]:
def day13_part1(inputs: list[str]) -> int:
    arrival_time = earliest_departure = int(inputs[0])
    bus_ids = [int(b) for b in inputs[1].split(',') if b != 'x']
    
    while True:
        for bus_id in bus_ids:
            if earliest_departure % bus_id == 0:
                return bus_id * (earliest_departure - arrival_time)
        earliest_departure += 1


def day13_part2(inputs: list[str]) -> int:  
    bus_id_order = [(int(b), i) for i, b in enumerate(inputs[1].split(',')) if b != 'x']
    mult =  bus_id_order[0][0] 
    t = 0  
    
    for bus_id, idx in bus_id_order[1:]:
        while (t + idx) % bus_id != 0:
            t += mult
        mult *= bus_id
    return t


run(day=13, parser=str, sep='\n', funcs=[day13_part1, day13_part2], example=True, actual=True)

Part 1 (example): 295    	Part 2 (example): 1068781
Part 1 (actual): 2095    	Part 2 (actual): 598411311431841


In [29]:
def day13_part1b(inputs: list[str]) -> int:  # O(n)
    ts = int(inputs[0])
    bus_ids = [int(b) for b in inputs[1].split(',') if b != 'x']
    
    min_bus_id = bus_ids[0]
    min_time = (ts // min_bus_id + 1) * min_bus_id
    
    for bus_id in bus_ids[1:]:
        candidate_min_time = (ts // bus_id + 1) * bus_id
        if candidate_min_time < min_time:
            min_time = candidate_min_time
            min_bus_id = bus_id
    return min_bus_id * (min_time - ts)
        

test_alternatives(day=13, funcs=[day13_part1, day13_part1b], runs=1000, parser=str)

Func Result:	2095  2095  
Func Runtime:	0.004  0.003  

In [30]:
def day13_part2b(inputs: list[str]) -> int:
    from z3 import Solver, Int, sat  #pip install z3-solver
    
    T, N = Int('T'), Int('N')
    t = 0
    
    bus_id_order = [(int(b), i) for i, b in enumerate(inputs[1].split(',')) if b != 'x']
    mult =  bus_id_order[0][0]   
    
    for bus_id, idx in bus_id_order[1:]:
        solver = Solver()  # build solver then add equations
        solver.add(T > 0)
        solver.add((T + idx) % bus_id == 0)
        solver.add(T == t + N * mult) # iterative condition
        assert solver.check() == sat  # returns solver is satisfiable
        t = solver.model()[T]
        mult *= bus_id
    return t


def day13_part2c(inputs: list[str]) -> int:
    from sympy.ntheory.modular import crt  # Chinese remainder theorm
    bus_ids = [int(b) for b in inputs[1].split(',') if b != 'x']
    offsets = [-i for  i, b in enumerate(inputs[1].split(',')) if b != 'x']
    ret, _ = crt(bus_ids, offsets)
    return ret
    

test_alternatives(day=13, funcs=[day13_part2, day13_part2b, day13_part2c], runs=10, parser=str)

Func Result:	598411311431841  598411311431841  598411311431841  
Func Runtime:	0.000  0.803  0.000  

## Day 14
- Input is a 'bit mask' followed by a series of memory assignments

- Part 1: Using the mask ignore 'X's, 1s and 0s set bits in the values being assigned to memory. Return the sum of all memory assigned.
- Part 2: Using the mask ignore 0's, 1's set bits in the values being assigned to memory, 'X's are both 0's and 1's (cycled through), set result to those bits and apply to the memory addresses (mulitple assignments). Return the sum of all memory assigned.

In [40]:
def day14_part1(text: str) -> int:
    memory = {}
    
    # 'no operations', as in 2's complement or with 0 / and with -1 has no effect
    # mask_or, mask_and  = 0, -1

    PATTERN = re.compile(r'^mem\[(\d+)\] = (\d+)$')
    for line in text.splitlines():
        if line.startswith('mask'):
            _, _, mask_s = line.partition(' = ')
            mask_or  = int(mask_s.replace('X', '0'), 2)
            mask_and = int(mask_s.replace('X', '1'), 2)
        else:
            match = PATTERN.match(line)
            assert match is not None, line
            target, value = int(match[1]), int(match[2])
            memory[target] = (value | mask_or) & mask_and
    return sum(memory.values())


def day14_part2(text: str) -> int:
    class Mask(NamedTuple):
        ones_mask: int
        x_masks: tuple[tuple[int, int], ...]

        def targets(self, number: int) -> Generator[int, None, None]:
            number = number | self.ones_mask
            for x_mask_or, x_mask_and in self.x_masks:
                yield (number | x_mask_or) & x_mask_and


    def parse_mask(s: str) -> Mask:
        one_mask = int(s.replace('X', '0'), 2)
        xs = [len(s) - 1 - match.start() for match in re.finditer('X', s)]
        x_masks = []
        for i in range(1 << len(xs)):
            number_or, number_and = 0, -1
            for j in range(len(xs)):
                bit = (i & (1 << j)) >> j  # could replace >> j with bool()
                if bit:
                    number_or |= 1 << xs[j]
                else:
                    number_and &= ~(1 << xs[j])
            x_masks.append((number_or, number_and))
        return Mask(one_mask, tuple(x_masks))
    
    
    memory = {}
    # mask = Mask(0, ())  # initialize dummy mask for type checker
    
    PATTERN = re.compile(r'^mem\[(\d+)\] = (\d+)$')
    for line in text.splitlines():
        if line.startswith('mask'):
            _, _, mask_s = line.partition(' = ')
            mask = parse_mask(mask_s)
        else:
            match = PATTERN.match(line)
            assert match is not None, line
            target, value = int(match[1]), int(match[2])
            for t in mask.targets(target):
                memory[t] = value
    return sum(memory.values())


run(day=14, parser=None, sep=None, funcs=[day14_part1, day14_part2], example=True, actual=True)

Part 1 (example): 51     	Part 2 (example): 208    
Part 1 (actual): 9296748256641	Part 2 (actual): 4877695371685


## Day 15

Input is a list of numbers used to seed the elves number game. 

After the first series of numbers, use these rules to determine the next numbers:

1) if the previous number is new then 0
2) else the next number is the difference between the previous turn (index/posistion) and the last turn the number was observed (i.e. [0,3,6,0] -> 4 - 1 == 3)
  
---
- Part 1: what is the 2020th number?
- Part 2: what is the 30,000,000th number?

In [42]:
# def day15_helper(nums: list[int], n: int) -> int:
#     # don't need to store all idx/pos, micro optimisation could use 
#     # defaultdict(lambda: deque(maxlen=2)), though actually slower in practice in python 3.10.8
#     seen = defaultdict(list)  
    
#     for turn in range(n):
#         if turn < len(nums):
#             n = nums[turn]
#         elif len(seen[n]) == 1:
#             n = 0
#         else:
#             n = seen[n][-1] - seen[n][-2]
#         seen[n].append(turn)
#     return n

def day15_helper(nums: list[int], n: int) -> int:
    last = nums[-1]
    seen = {num: idx for idx, num in enumerate(nums, 1)}
    for idx in range(len(nums), n):
        seen[last], last = idx, idx - seen.get(last, idx)
    return last


def day15_part1(nums: list[int]) -> int:
    return day15_helper(nums, 2020)


def day15_part2(nums: list[int]) -> int:
    return day15_helper(nums, 30_000_000)


run(day=15, parser=int, sep=',', funcs=[day15_part1, day15_part2], example=True, actual=True)

Part 1 (example): 1836   	Part 2 (example): 362    
Part 1 (actual): 755     	Part 2 (actual): 11962   


Part 1 (example): 1836   	Part 2 (example): 362    
Part 1 (actual): 755     	Part 2 (actual): 11962   


## Day 16: Ticket Translation
Input: series of valid integer ranges for ticket fields, your ticket, and nearby tickets.
- Part 1: return the sum of all fields not valid for any field
- Part 2: discard nearby tickets with an invalid field and find which fields map to which slot, return the product of all fields starting with 'depature'

In [35]:
rules = dict[str, tuple[int, int, int, int]]
ticket = list[int]


def day16_parser(inputs: list[str, str, str]) -> tuple[rules, ticket, list[ticket]]:
    r, t, nt = (i.split('\n') for i in inputs)  # split three sections
    
    rule_re = re.compile(r'^([^:]+): (\d+)-(\d+) or (\d+)-(\d+)$')
    rules = {}
    for line in r:
        m = rule_re.match(line)
        assert m is not None, line
        rules[m[1]] = (int(m[2]), int(m[3]), int(m[4]), int(m[5]))
    
    parse_ticket = lambda s: list(map(int, s.split(',')))
    ticket = parse_ticket(t[1])  # ignore heading 'your ticket:'
    nearby_tickets = [parse_ticket(i) for i in nt[1:]]  # ignore heading 'nearby tickets:'
    return rules, ticket, nearby_tickets


def day16_part1(inputs: list[str, str, str]) -> int:
    rules, _, nearby_tickets = day16_parser(inputs)
    total = 0
    for nt in nearby_tickets:
        for num in nt:  # start / end ranges (of allowed values)
            for s1, e1, s2, e2 in rules.values():  
                if s1 <= num <= e1 or s2 <= num <=e2:
                    break
            else:  # only happens if all are invalid
                total += num
    return total


def day16_part2(inputs: list[str, str, str]) -> int:
    rules, ticket, nearby_tickets = day16_parser(inputs)
    valid_tickets = []
    for nt in nearby_tickets:
        if all(
            any(
                s1 <= num <= e1 or s2 <= num <= e2
                for s1, e1, s2, e2 in rules.values()
            )
            for num in nt
        ):
            valid_tickets.append(nt)

    possible_at_pos = defaultdict(set)
    for pos in range(len(valid_tickets[0])):
        for key, (s1, e1, s2, e2) in rules.items():
            for vt in valid_tickets:
                if not(s1 <= vt[pos] <= e1 or s2 <= vt[pos] <= e2):
                    break
            else:   # all tickets passed 
                possible_at_pos[pos].add(key)
    
    positions = {}
    while possible_at_pos:
        for k, v in tuple(possible_at_pos.items()):
            if len(v) == 1:
                key, = v
                positions[key] = k
                possible_at_pos.pop(k)
                for v in possible_at_pos.values():
                    v.discard(key)
                
    ret = 1
    for k,v in positions.items():
        if k.startswith('departure'):
            ret *= ticket[v]
    return ret


run(day=16, parser=str, sep='\n\n', funcs=[day16_part1, day16_part2], example=True, actual=True)

Part 1 (example): 71     	Part 2 (example): 1      
Part 1 (actual): 21956   	Part 2 (actual): 3709435214239


# Doing

## Day 17

In [36]:
def day17_part1(text: str) -> int:
    activated = {
        (0, y, x)
        for y, line in enumerate(text.splitlines())
        for x, char in enumerate(line)
        if char == '#'  # filter inactive char '.'
    }
    
    for _ in range(6):
        marked: Counter(tuple(int, int, int)) = Counter()
        for z, y, x in activated:
            for z_i in (-1, 0, 1):
                for y_i in (-1, 0, 1):
                    for x_i in (-1, 0, 1):
                        if z_i == y_i == x_i == 0:
                            continue
                        idx = (z + z_i, y + y_i, x + x_i)
                        marked[idx] += 1
                        
        activated = {k for k in activated if marked[k] == 2}
        for k, v in marked.items():
            if v == 3:
                activated.add(k)
    return len(activated)


def day17_part2(text: str) -> int:
    activated =  {
        (0, 0, y, x)
        for y, line in enumerate(text.splitlines())
        for x, char in enumerate(line)
        if char == '#'  # filter inactive char '.'
    }

    for _ in range(6):
        marked: Counter(tuple(int, int, int, int)) = Counter()
        for w, z, y, x in activated:
            for w_i in (-1, 0, 1):
                for z_i in (-1, 0, 1):
                    for y_i in (-1, 0, 1):
                        for x_i in (-1, 0, 1):
                            if w_i == z_i == y_i == x_i == 0:
                                continue
                            idx = (w + w_i, z + z_i, y + y_i, x + x_i)
                            marked[idx] += 1
                        
        activated = {k for k in activated if marked[k] == 2}
        for k, v in marked.items():
            if v == 3:
                activated.add(k)
    return len(activated)


run(day=17, parser=None, sep=None, funcs=[day17_part1, day17_part2], example=True, actual=True)

Part 1 (example): 112    	Part 2 (example): 848    
Part 1 (actual): 313     	Part 2 (actual): 2640    


In [37]:
from typing import Match

def parens_sub_callback(match: Match[str]) -> str:
    def compute(s: str) -> int:
        parts = s.split()
        value = int(parts[0])
        i = 1
        while i < 
    return str(compute(match[0])) 
    

PARENS = re.compile(r'\(([^()]+)\)')


def day18_part1(text: str) -> int:
    total = 0
    for line in text.splitlines():
        while PARENS.search(line):
            PARENS.sub(parens_sub_callback, line)
            
    return total


def day18_part2(policies: list[tuple]) -> int:
    pass

run(day=18, parser=day18_parser, sep='\n', funcs=[day18_part1, day17_part2], example=True, actual=False)

SyntaxError: invalid syntax (3581511928.py, line 8)

# Rnd

In [ ]:
a = '2 * 3 + (4 * 5)'.replace(' ', '')



res = 0
ops = deque(maxlen=3)

for i in a:
    print(ops)
    if len(ops) == 3:
        res = eval(''.join(sorted(map(str, ops))))
        ops.append(res)
    else:
        ops.append(i)
print(res)

In [ ]:
import ast

ast.literal_eval

# Python doesn't have the idea of a sign on an integer, so (neg. nums are) parsed as a unary minus:
ast.dump(ast.parse('-3').body[0])

In [ ]:
# js https://davidlozzi.com/2020/12/09/advent-of-code-day-9/
# https://dev.to/qviper/advent-of-code-2020-python-solution-day-9-1knjß
# https://dev.to/rpalo/advent-of-code-2020-solution-megathread-day-9-encoding-error-594o

In [ ]:
## TODO review Part 14 Part 2
## TODO review Part 11 Part 
## TODO review Part 16 Part 2 